In [16]:
import os
import pandas as pd
import db_dtypes
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/Users/alejandrogiraldoh/Downloads/mymoneyapp-367411-9b9c5c3455d8.json'

In [3]:
from google.cloud import bigquery

In [4]:
client = bigquery.Client()

In [5]:
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [6]:
# Show all the tables in the Stack OverFlow dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset
for table in tables:  
    print(table.table_id)

badges
comments
post_history
post_links
posts_answers
posts_moderator_nomination
posts_orphaned_tag_wiki
posts_privilege_wiki
posts_questions
posts_tag_wiki
posts_tag_wiki_excerpt
posts_wiki_placeholder
stackoverflow_posts
tags
users
votes


In [7]:
# Construct a reference to the "badges" table
table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
table = client.get_table(table_ref)

In [8]:
# Print information on all the columns in the "badges" table in the "Stack Overflow" dataset
table.schema

[SchemaField('id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('title', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('body', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('accepted_answer_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('answer_count', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('comment_count', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('community_owned_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('creation_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('favorite_count', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('last_activity_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('last_edit_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('last_editor_display_name', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('last_editor_user_id', 'INTEGER', 'NULLABLE', None, None, (), None),


In [ ]:
table = "tags"
query = f"select * from bigquery-public-data.stackoverflow.{table}"

df = client.query(query).to_dataframe()

/Users/alejandrogiraldoh/dev/daverse/stack-overflow-analytics/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63653 entries, 0 to 63652
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               63653 non-null  Int64 
 1   tag_name         63653 non-null  object
 2   count            63653 non-null  Int64 
 3   excerpt_post_id  44966 non-null  Int64 
 4   wiki_post_id     44966 non-null  Int64 
dtypes: Int64(4), object(1)
memory usage: 2.7+ MB


In [27]:
table = pa.Table.from_pandas(df)
pq.write_table(table, 'example.parquet', compression='snappy', row_group_size=100000)

In [ ]:
import boto3

# Instantiate AWS client for S3
session = boto3.Session(profile_name="daverse-dev")
s3_client = session.client("s3")


In [36]:
# Example: List the buckets to verify the client works
buckets = s3_client.list_buckets()
print("Buckets:", [bucket["Name"] for bucket in buckets["Buckets"]])

Buckets: ['almadosreiss-dev-product-images', 'cdk-hnb659fds-assets-992382719587-us-east-1']


In [41]:
# Create AWS S3 bucket
region = 'us-east-1'
s3_client.create_bucket(Bucket='stackoverflow-analytics-datasets')

{'ResponseMetadata': {'RequestId': 'BND23BTZ3GQR1Q75',
  'HostId': 'i6N/Bn6VTBrpUqyJ3XvbYzI/P5Ra0kB7QwTEgFA6osWQrHVMdb9F4RD3APK5ocExSEeN4chgo7Q=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'i6N/Bn6VTBrpUqyJ3XvbYzI/P5Ra0kB7QwTEgFA6osWQrHVMdb9F4RD3APK5ocExSEeN4chgo7Q=',
   'x-amz-request-id': 'BND23BTZ3GQR1Q75',
   'date': 'Fri, 15 Nov 2024 00:38:58 GMT',
   'location': '/stackoverflow-analytics-datasets',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/stackoverflow-analytics-datasets'}

In [45]:
# Upload file into S3
file_name = 'example.parquet'
s3_client.upload_file(file_name, 'stackoverflow-analytics-datasets', 'example.parquet')

In [29]:
# Read Parquet file
parquet_table = pq.read_table('example.parquet')

# Convert Parquet table to DataFrame
df = parquet_table.to_pandas()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63653 entries, 0 to 63652
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               63653 non-null  Int64 
 1   tag_name         63653 non-null  object
 2   count            63653 non-null  Int64 
 3   excerpt_post_id  44966 non-null  Int64 
 4   wiki_post_id     44966 non-null  Int64 
dtypes: Int64(4), object(1)
memory usage: 2.7+ MB
